# Scikit-Learn IRIS Model

 * Wrap a scikit-learn python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on a kubernetes cluster
 
## Dependencies

 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install sklearn
pip install seldon-core
```

## Train locally
 

In [ ]:
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn import datasets

def main():
    clf = LogisticRegression()
    p = Pipeline([('clf', clf)])
    print('Training model...')
    p.fit(X, y)
    print('Model trained!')

    filename_p = 'IrisClassifier.sav'
    print('Saving model in %s' % filename_p)
    joblib.dump(p, filename_p)
    print('Model saved!')
    
if __name__ == "__main__":
    print('Loading iris data set...')
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print('Dataset loaded!')
    main()


Wrap model using s2i

## REST test

In [ ]:
!s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.18 seldonio/sklearn-iris:0.1

In [ ]:
!docker run --name "iris_predictor" -d --rm -p 5000:5000 seldonio/sklearn-iris:0.1

Send some random features that conform to the contract

In [ ]:
!curl  -s http://localhost:5000/predict -H "Content-Type: application/json" -d '{"data":{"ndarray":[[5.964,4.006,2.081,1.031]]}}'

In [ ]:
!docker rm iris_predictor --force

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb) to setup Seldon Core with an ingress - either Ambassador or Istio.

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: `kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080`
 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:8080`

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
!kubectl create -f sklearn_iris_deployment.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-deployment-example \
                                 -o jsonpath='{.items[0].metadata.name}')

In [ ]:
for i in range(60):
    state=!kubectl get sdep seldon-deployment-example -o jsonpath='{.status.state}'
    state=state[0]
    print(state)
    if state=="Available":
        break
    time.sleep(1)
assert(state=="Available")

In [ ]:
res=!curl  -s http://localhost:8003/seldon/seldon/seldon-deployment-example/api/v0.1/predictions -H "Content-Type: application/json" -d '{"data":{"ndarray":[[5.964,4.006,2.081,1.031]]}}'

In [ ]:
res

In [ ]:
print(res)
import json
j=json.loads(res[0])
assert(j["data"]["ndarray"][0][0]>0.0)

In [ ]:
!kubectl delete -f sklearn_iris_deployment.yaml